# Sequence Generation using IndoGPT

In [1]:
import os, sys
sys.path.append('../')
os.chdir('../')
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

import torch
import shutil
import random
import numpy as np
import pandas as pd
from torch import optim
from transformers import GPT2LMHeadModel

from indobenchmark import IndoNLGTokenizer
from utils.train_eval import train, evaluate
from utils.metrics import generation_metrics_fn
from utils.forward_fn import forward_generation
from utils.data_utils import MachineTranslationDataset, GenerationDataLoader

In [2]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
# Set random seed
set_seed(26092020)

# Load Model

In [3]:
gpt_model = GPT2LMHeadModel.from_pretrained('indobenchmark/indogpt')
tokenizer = IndoNLGTokenizer.from_pretrained('indobenchmark/indogpt')
model = gpt_model
model

Downloading:   0%|          | 0.00/707 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/602M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/303 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(40005, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [4]:
count_param(model)

116566272

# Test Generation

In [6]:
gpt_input = torch.LongTensor([tokenizer.encode('<s> aku adalah anak', add_special_tokens=False)])
gpt_out = gpt_model.generate(gpt_input, do_sample=True, pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id)
tokenizer.decode(gpt_out[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<s> aku adalah anak pertama dari sepasang suami istri yang lahir pada tanggal 27 agustus.</s> tahun 2009,'

In [17]:
gpt_input = torch.LongTensor([tokenizer.encode('<s> hai, bagaimana kabar', add_special_tokens=False)])
gpt_out = gpt_model.generate(gpt_input, do_sample=True, pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id)
tokenizer.decode(gpt_out[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<s> hai, bagaimana kabar anda hari ini? semoga anda berada dalam keadaan sehat dan baik-baik saja'

In [28]:
gpt_input = torch.LongTensor([tokenizer.encode('<s>', add_special_tokens=False)])
gpt_out = gpt_model.generate(gpt_input, do_sample=True, min_length=200, max_length=200, 
    pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id)
tokenizer.decode(gpt_out[0], skip_special_tokens=True)

' dalam rangka menyambut hut republik indonesia ke 73 tahun, maka tanggal 14-19 juli 2016 kantor perpustakaan dan arsip daerah kabupaten kulon progo menyelenggarakan kegiatan festival literasi di smp dan sma muhammadiyah wates, jl. ir. h. djuanda no. 8, wates yogyakarta. acara festival literasi ini diikuti oleh seluruh siswa smp dan sma muhammadiyah wates, serta perwakilan sd/mi negeri, smp/mts swasta, dan sma muhammadiyah wates, serta siswa-siswa kelas 9 (delapan belas) dan 12 (sepuluh belas) dari smp/mts swasta, serta siswa-siswi sd negeri dan sma swasta, baik negeri maupun swasta yang mengikuti acara festival literasi tingkat kab. kulon progo dan diy. kegiatan lomba ini diselenggarakan setiap tahun. pada tahun 2016 ini smp negeri 3 kedungwuni yogyakarta mengirimkan 20 siswa siswa-siswi berprestasi tingkat kab. kulon progo, sebagai berikut : juara 1 lomba baca puisi tingkat kabupaten kulon progo (smu) juara 3 lomba baca puisi tingkat kabupaten kulon progo (smpl'